# 地理信息系统设计与开发(shp文件的投影)

In [18]:
import os
import ogr
import osr
import json
import requests
import matplotlib.pyplot as plt
%matplotlib inline

## 读入一个数据
## 获取这个数据的空间参考
## 设定要转换的坐标系统
## 获取坐标系统之间的转换参数
## 新建一个图层用于保存坐标转换后的数据
## 读取出数据中的每一个feature，通过geometry的Transform进行坐标转换
## 将转换后的结果写入新的图层

In [19]:
driver=ogr.GetDriverByName('ESRI Shapefile')
dataSource=driver.Open('yunnan.shp')
inputLayer=dataSource.GetLayer()

In [20]:
inSpatialRef=inputLayer.GetSpatialRef()
print(inSpatialRef)

GEOGCS["WGS 84",
    DATUM["WGS_1984",
        SPHEROID["WGS 84",6378137,298.257223563,
            AUTHORITY["EPSG","7030"]],
        AUTHORITY["EPSG","6326"]],
    PRIMEM["Greenwich",0,
        AUTHORITY["EPSG","8901"]],
    UNIT["degree",0.0174532925199433,
        AUTHORITY["EPSG","9122"]],
    AXIS["Latitude",NORTH],
    AXIS["Longitude",EAST],
    AUTHORITY["EPSG","4326"]]


In [21]:
outSpatialRef=osr.SpatialReference()
outSpatialRef.ImportFromEPSG(32648)
print(outSpatialRef)

PROJCS["WGS 84 / UTM zone 48N",
    GEOGCS["WGS 84",
        DATUM["WGS_1984",
            SPHEROID["WGS 84",6378137,298.257223563,
                AUTHORITY["EPSG","7030"]],
            AUTHORITY["EPSG","6326"]],
        PRIMEM["Greenwich",0,
            AUTHORITY["EPSG","8901"]],
        UNIT["degree",0.0174532925199433,
            AUTHORITY["EPSG","9122"]],
        AUTHORITY["EPSG","4326"]],
    PROJECTION["Transverse_Mercator"],
    PARAMETER["latitude_of_origin",0],
    PARAMETER["central_meridian",105],
    PARAMETER["scale_factor",0.9996],
    PARAMETER["false_easting",500000],
    PARAMETER["false_northing",0],
    UNIT["metre",1,
        AUTHORITY["EPSG","9001"]],
    AXIS["Easting",EAST],
    AXIS["Northing",NORTH],
    AUTHORITY["EPSG","32648"]]


In [22]:
transform=osr.CoordinateTransformation(inSpatialRef,outSpatialRef)
print(transform)

<osgeo.osr.CoordinateTransformation; proxy of <Swig Object of type 'OSRCoordinateTransformationShadow *' at 0x000002745F6E3030> >


In [23]:
outFileName='myReprojectShp1.shp'
if os.path.exists(outFileName):
    driver.DeleteDataSource(outFileName)
outDataSource=driver.CreateDataSource(outFileName)
outLayer = outDataSource.CreateLayer("yunnan_32648", outSpatialRef, geom_type=ogr.wkbPolygon)

In [24]:
inFeatureDefn = inputLayer.GetLayerDefn()

In [26]:
for feat in inputLayer:
    geom=feat.geometry()
    #print("old geom:",geom)
    geom.Transform(transform)
    #print("new geom:",geom)
    
    feature=ogr.Feature(inFeatureDefn)
    feature.SetGeometry(geom)
    outLayer.CreateFeature(feature)
    
    feature=None
    
dataSource=None
outDataSource=None